# Extracting TotalGray From SUB 1

In [63]:
file_path = r"Z:\Pain_Data\preproc\sourcedata\freesurfer\sub-01\stats\aseg.stats"

# Initialize variable to store TotalGray value
total_gray = None

# Open the file and search for the TotalGray line
with open(file_path, 'r') as file:
    for line in file:
        if "Measure TotalGray," in line:
            # Extract the fourth column (value of TotalGray)
            total_gray = line.split(",")[3].strip()
            break

if total_gray:
    print(f"TotalGray value: {total_gray}")
else:
    print("TotalGray not found in the file.")

TotalGray value: 539130.558278


In [ ]:
temp

# Imports

In [64]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold


# Extracting TotalGray From All Subjects

In [65]:


def extract_gray_matter_volume(aseg_stats_path):
    # Initialize variable to store TotalGray value
    total_gray = None

# Open the file and search for the TotalGray line
    with open(aseg_stats_path, 'r') as file:
        for line in file:
            if "Measure TotalGray," in line:
                # Extract the fourth column (value of TotalGray)
                total_gray = line.split(",")[3].strip()
                break

    if total_gray:
        # print(f"TotalGray value: {total_gray}")
        return total_gray
    else:
        # print("TotalGray not found in the file.")
        return None
    
# Extract gray matter volumes for all subjects
base_dir = r"Z:\Pain_Data\preproc\sourcedata\freesurfer"  # Base directory for FreeSurfer data

subject_dirs = [
    os.path.join(base_dir, sub, 'stats', 'aseg.stats')
    for sub in os.listdir(base_dir)
    if os.path.isdir(os.path.join(base_dir, sub))
]
subject_dirs = subject_dirs[1:] 

subject_TotalGray_data = []

for aseg_path in subject_dirs:
    subject_id = os.path.basename(os.path.dirname(os.path.dirname(aseg_path)))
    gray_matter_volume = extract_gray_matter_volume(aseg_path)
    if gray_matter_volume is not None:
        subject_TotalGray_data.append({'Subject no.': subject_id, 'GrayMatterVolume': gray_matter_volume})

gray_matter_df = pd.DataFrame(subject_TotalGray_data)


# Extracting Patient Data

In [66]:
# Step 3: Load demographic data
Patient_Data_path = r"C:\Users\zevel\Downloads\התנסות מחקרית\Copy of data_Pre_Post_Placebo_paper_2016_Tetreault_P (2).csv"  # Update with your file path
Patient_Data = pd.read_csv(Patient_Data_path)

# Step 4: Merge gray matter data with demographics using "Subject no."
merged_data = pd.merge(Patient_Data, gray_matter_df, on="Subject no.", how="inner")






# Encoding the Variables

In [67]:
# Categorize subjects into "Control" and "Chronic Pain" groups
def assign_group(subject_no):
    subject_number = int(subject_no.split('-')[1])  # Extract the numeric part of Subject no.
    return "Control" if subject_number <= 20 else "Chronic Pain"

merged_data['PainGroup'] = merged_data['Subject no.'].apply(assign_group) # New column

# Encode categorical variables (e.g., Gender and PainGroup)
label_encoder = LabelEncoder()
# merged_data['GenderEncoded'] = label_encoder.fit_transform(merged_data['gender']) # No Need
merged_data['PainGroupEncoded'] = merged_data['PainGroup'].map({"Control": 0, "Chronic Pain": 1})




# Preparing Data for Regression

In [68]:
X = merged_data[['age', 'gender 01', 'PainGroupEncoded']]  # Predictors
y = merged_data['GrayMatterVolume']  # Target variable

# Train-Test split 
This is an example for our understanding, these variables are not used in the cross validation. There it is done again

In [69]:
# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train-dev split (from the remaining 80% which is X_train_full)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42) #this is done again in the cross validation

# Print sizes to verify the splits
print(f"Train size: {len(X_train)}")
print(f"Dev size: {len(X_dev)}")
print(f"Test size: {len(X_test)}")

Train size: 48
Dev size: 12
Test size: 16


# Cross Validation on Train subjects

In [70]:
# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Number of folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) # Calling KFold



# Convert X_train  and Y_train to a NumPy array
X_train_np = X_train_full.to_numpy()
y_train_np = y_train_full.to_numpy()

r2_scores = []
mse_scores = []

# Cross-validation
for fold, (train_index, dev_index) in enumerate(kf.split(X_train_np)):
    # Split into training and validation sets
    X_train_fold, X_dev_fold = X_train_np[train_index], X_train_np[dev_index]
    y_train_fold, y_dev_fold = y_train_np[train_index], y_train_np[dev_index]
    
    # Train the model
    model = LinearRegression()  
    model.fit(X_train_fold, y_train_fold)
    
    # Predict on the validation set
    y_dev_pred = model.predict(X_dev_fold)
    
    # Compute metrics
    mse = mean_squared_error(y_dev_fold, y_dev_pred)
    r2 = r2_score(y_dev_fold, y_dev_pred)
    
    # Append metrics to the lists
    mse_scores.append(mse)
    r2_scores.append(r2)
    
    print(f"Fold {fold+1}: MSE = {mse:.2f}, R^2 = {r2:.2f}")

# Calculate average metrics
avg_mse = sum(mse_scores) / len(mse_scores)
avg_r2 = sum(r2_scores) / len(r2_scores)

print("\nCross-Validation Performance:")
print(f"Average MSE: {avg_mse:.2f}")
print(f"Average R^2: {avg_r2:.2f}")

Fold 1: MSE = 1563721952.88, R^2 = 0.52
Fold 2: MSE = 2602391273.14, R^2 = 0.19
Fold 3: MSE = 2059310348.40, R^2 = -0.12
Fold 4: MSE = 2133391386.08, R^2 = 0.28
Fold 5: MSE = 4444450461.02, R^2 = -0.15

Cross-Validation Performance:
Average MSE: 2560653084.30
Average R^2: 0.14


# Evaluating the Final Model on X_test, y_test

In [76]:
# Train the final model on the entire training set
final_model = LinearRegression()  
final_model.fit(X_train_full, y_train_full)

# Predict on the test set
y_test_pred = final_model.predict(X_test)

# Evaluate the final model on the test set
from sklearn.metrics import mean_squared_error, r2_score
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print("\nTest Set Performance:")
print(f"Test MSE: {mse_test:.2f}")
print(f"Test R^2: {r2_test:.2f}")
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# Finding feature importance
print("\nFeature Importance:")
for feature, coef in zip(X.columns, model.coef_):
    print(f"{feature}: {coef:.3f}")


Test Set Performance:
Test MSE: 1493661581.16
Test R^2: -0.02
Coefficients: [  -281.24550226  62244.95406869 -25321.76342007]
Intercept: 558455.2314730741

Feature Importance:
age: -281.246
gender 01: 62244.954
PainGroupEncoded: -25321.763


# Turning All Of This Into a Function

In [84]:
def Lin_Regress(x_cols):
    
    print(f"\nRegressors: {x_cols} ")
    print()

    
    X = merged_data[x_cols]
    y = merged_data['GrayMatterVolume']  # Target variable
    


    # Train-test split
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Number of folds
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) # Calling KFold

    # Convert X_train  and Y_train to a NumPy array
    X_train_np = X_train_full.to_numpy()
    y_train_np = y_train_full.to_numpy()

    r2_scores = []
    mse_scores = []

    # Cross-validation
    for fold, (train_index, dev_index) in enumerate(kf.split(X_train_np)):
        # Split into training and validation sets
        X_train_fold, X_dev_fold = X_train_np[train_index], X_train_np[dev_index]
        y_train_fold, y_dev_fold = y_train_np[train_index], y_train_np[dev_index]
        
        # Train the model
        model = LinearRegression()  
        model.fit(X_train_fold, y_train_fold)
        
        # Predict on the validation set
        y_dev_pred = model.predict(X_dev_fold)
        
        # Compute metrics
        mse = mean_squared_error(y_dev_fold, y_dev_pred)
        r2 = r2_score(y_dev_fold, y_dev_pred)
        
        # Append metrics to the lists
        mse_scores.append(mse)
        r2_scores.append(r2)
        
        print(f"Fold {fold+1}: MSE = {mse:.2f}, R^2 = {r2:.2f}")

    # Calculate average metrics
    avg_mse = sum(mse_scores) / len(mse_scores)
    avg_r2 = sum(r2_scores) / len(r2_scores)

    print("\nCross-Validation Performance:")
    print(f"Average MSE: {avg_mse:.2f}")
    print(f"Average R^2: {avg_r2:.2f}")
    
    # Train the final model on the entire training set
    final_model = LinearRegression() 
    final_model.fit(X_train_full, y_train_full)

    # Predict on the test set
    y_test_pred = final_model.predict(X_test)

    # Evaluate the final model on the test set
    #from sklearn.metrics import mean_squared_error, r2_score
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)

    print("\nTest Set Performance:")
    print(f"Test MSE: {mse_test:.2f}")
    print(f"Test R^2: {r2_test:.2f}")
    print("Coefficients", model.coef_)
    print("Intercept:", model.intercept_)
    
    return mse_test, r2_test, model.coef_, model.intercept_



# Performing Linear Regression for each Variable Seperately

In [85]:
Lin_Regress(['PainGroupEncoded'])
Lin_Regress(['age'])
Lin_Regress(['gender 01'])
Lin_Regress(['age', 'gender 01', 'PainGroupEncoded'])




Regressors: ['PainGroupEncoded'] 

Fold 1: MSE = 2894488427.79, R^2 = 0.12
Fold 2: MSE = 4022202003.01, R^2 = -0.26
Fold 3: MSE = 1891869863.79, R^2 = -0.03
Fold 4: MSE = 3227040636.42, R^2 = -0.09
Fold 5: MSE = 4322435965.77, R^2 = -0.12

Cross-Validation Performance:
Average MSE: 3271607379.36
Average R^2: -0.08

Test Set Performance:
Test MSE: 1754212176.99
Test R^2: -0.20
Coefficients [-26641.76801256]
Intercept: 575659.5060713848

Regressors: ['age'] 

Fold 1: MSE = 3754581404.80, R^2 = -0.15
Fold 2: MSE = 3371432940.37, R^2 = -0.05
Fold 3: MSE = 1717257672.49, R^2 = 0.07
Fold 4: MSE = 3291256509.75, R^2 = -0.12
Fold 5: MSE = 4405664652.96, R^2 = -0.14

Cross-Validation Performance:
Average MSE: 3308038636.07
Average R^2: -0.08

Test Set Performance:
Test MSE: 1725339007.67
Test R^2: -0.18
Coefficients [-991.41357598]
Intercept: 613260.1519639702

Regressors: ['gender 01'] 

Fold 1: MSE = 1591104099.34, R^2 = 0.51
Fold 2: MSE = 2348954758.14, R^2 = 0.27
Fold 3: MSE = 2395012134.2

(1493661581.1619155,
 -0.01801689237436821,
 array([  -281.24550226,  62244.95406869, -25321.76342007]),
 np.float64(558455.2314730741))